In [5]:
!nvidia-sml

/bin/bash: nvidia-sml: command not found


In [6]:
!pip install nvidia

ERROR: Could not find a version that satisfies the requirement nvidia (from versions: none)
ERROR: No matching distribution found for nvidia


In [7]:
from __future__ import  absolute_import,print_function,division,unicode_literals

try:
  !pip install tb-nightly tesnsorboardX tensorboard
  !pip install tf-nightly
except Exception:
  pass

import tensorflow as tf
import os
import datetime
import tensorflow_datasets as tfds
%load_ext tensorboard

ERROR: Could not find a version that satisfies the requirement tesnsorboardX (from versions: none)
ERROR: No matching distribution found for tesnsorboardX
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
dataset,info=tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00',with_info=True)
train_dataset=dataset['train']

In [9]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [10]:
print(train_dataset)

<PrefetchDataset shapes: {data: {customer_id: (), helpful_votes: (), marketplace: (), product_category: (), product_id: (), product_parent: (), product_title: (), review_body: (), review_date: (), review_headline: (), review_id: (), star_rating: (), total_votes: (), verified_purchase: (), vine: ()}}, types: {data: {customer_id: tf.string, helpful_votes: tf.int32, marketplace: tf.string, product_category: tf.string, product_id: tf.string, product_parent: tf.string, product_title: tf.string, review_body: tf.string, review_date: tf.string, review_headline: tf.string, review_id: tf.string, star_rating: tf.int32, total_votes: tf.int32, verified_purchase: tf.int64, vine: tf.int64}}>


In [11]:
len(list(train_dataset))

104975

In [12]:
BUFFER_SIZE=30000
BATCH_SIZE=128

In [13]:
train_dataset=train_dataset.shuffle(BUFFER_SIZE,reshuffle_each_iteration=False)

In [14]:
for reviews in train_dataset.take(2):
  print(reviews)

{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'12272706'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Mobile_Electronics'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B00GLOWD3Q'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'854343314'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Excelvan F11B FM Transmitter+ Car Charger Holder + Remote Control Compatible with iPhone 5/5S/5C, iPod Touch 5 /iPod Nano 7'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b"Save yourself a headache, find an actual supported product from Apple. When you plug it in your phone says the device is not supported. I have the iPhone 5. This item was so hard to release from my cigarette lighter that I had to go to my mechanic because my whole console was going to come apart with anymore fo

In [15]:
for reviews in train_dataset.take(10):
  review_text=reviews['data']
  print(review_text.get('review_body').numpy())
  print(review_text.get('star_rating'))
  print(tf.where(review_text.get('star_rating')>3,1,0).numpy())

b"Save yourself a headache, find an actual supported product from Apple. When you plug it in your phone says the device is not supported. I have the iPhone 5. This item was so hard to release from my cigarette lighter that I had to go to my mechanic because my whole console was going to come apart with anymore force. The charge on your phone goes on and off when plugged in and beeps and can't be a good sign when you screen goes blank. Forget it."
tf.Tensor(1, shape=(), dtype=int32)
0
b"I have both cd's and cassettes and this lets me play either with just one machine.  I use it all the time and would recommend it to anyone."
tf.Tensor(5, shape=(), dtype=int32)
1
b'Crap, Crap and more crap.  The iphone side of the connector broke within a week or so after owning it.'
tf.Tensor(1, shape=(), dtype=int32)
0
b"Shipper was great.  Fast to work with.  But this product sucks and I would not spend the money.  I've heard they have fixed the issues in later vehicles.  This is a glorified iPhone ch

In [17]:
tokenizer=tfds.deprecated.text.Tokenizer()

vocabulary_set=set()

for _,reviews in train_dataset.enumerate():
  review_text=reviews['data']
  reviews_tokens=tokenizer.tokenize(review_text.get('review_body').numpy())
  vocabulary_set.update(reviews_tokens)

vocab_size=len(vocabulary_set)
vocab_size

73738

In [18]:
encoder=tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

In [19]:
print(vocabulary_set)

{'graphics', 'unscrews', 'aerobars', 'projectile', '5050', '1200wRMS', 'irreplaceable', 'haul', 'S401', 'flaking', 'Bullets', '450RMS', 'FAVORITE', 'magnetic', 'markings', 'preceding', 'omnidirectional', 'excursions', 'EInk', 'haad', 'REVEIWS', 'origimal', 'siginal', 'choppy', 'crock', 'Suitable', '100mm', 'meteor', 'avilaile', 'ANGRY', 'CarProCam', 'Ram', 'baratas', '12w6v2', 'mediation', 'mercury', 'Keyspan', 'qstarz', 'discrepancy', 'seach', 'P1000', 'relevant', 'Unusual', 'jukeboxes', 'sink', '533i', 'SKEPTICAL', '771', 'VARIETY', 'firms', 'operation', 'Hopefully', 'eras', 'everthingthing', 'micormeter', 'littlevaudio', 'EFORCITY', 'tube', '40LMs', 'anything', 'exaggerates', 'expelled', 'outsell', 'demás', 'imp1000', '10MP', 'humidity', 'XtraFlex2', 'bounty', '1999', 'T1s', 'actaul', 'Bionic', 'intensively', 'horrid', 'ruining', 'SJCam', 'departement', '20203R79903', 'SILVER', 'appletv', 'Deffitnely', 'distorted', 'notification', 'acertive', 'halt', 'COPYRIGHTED', 'fulfilled', 'dis

In [20]:
for reviews in train_dataset.take(5):
  review_text=reviews['data']
  print(review_text.get('review_body').numpy())
  encoded_example=encoder.encode(review_text.get('review_body').numpy())
  print(encoded_example)
  
# vocab_size=len(vocabulary_set)
# vocab_size

b"Save yourself a headache, find an actual supported product from Apple. When you plug it in your phone says the device is not supported. I have the iPhone 5. This item was so hard to release from my cigarette lighter that I had to go to my mechanic because my whole console was going to come apart with anymore force. The charge on your phone goes on and off when plugged in and beeps and can't be a good sign when you screen goes blank. Forget it."
[67446, 13311, 690, 41220, 15368, 42316, 30162, 62045, 47170, 32918, 34759, 16010, 39590, 5659, 35862, 37410, 61161, 16109, 41239, 20269, 13248, 28102, 72849, 62045, 23834, 23522, 20269, 72013, 34895, 10553, 13392, 66158, 57753, 31458, 19521, 46646, 32918, 37647, 35364, 34927, 12814, 23834, 22787, 19521, 9031, 19521, 37647, 52079, 60875, 37647, 17156, 65236, 66158, 72647, 19521, 43497, 50666, 36611, 70351, 15009, 32425, 25229, 67055, 61161, 16109, 52218, 67055, 6268, 62053, 25593, 4827, 37410, 6268, 45745, 6268, 27966, 35237, 55287, 690, 33308

In [21]:
for index in encoded_example:
  print('{}----->{}'.format(index,encoder.decode([index])))

10553----->This
31536----->speaker
28102----->is
32860----->very
72387----->cute
6268----->and
27966----->can
17143----->fill
690----->a
17156----->whole
20661----->room
36611----->with
9746----->sound
52327----->br
52327----->br
32425----->The
31536----->speaker
51366----->sounds
49060----->as
33308----->good
49060----->as
15366----->those
67055----->on
37647----->my
40984----->TV
8852----->It
24998----->s
10063----->reachargable
6268----->and
32860----->very
45314----->portable
69469----->There
24998----->s
52753----->really
15330----->no
44063----->way
19521----->to
2887----->tell
10006----->how
62440----->much
7747----->battery
28102----->is
43032----->left
37410----->in
20269----->the
31536----->speaker
64922----->or
72027----->if
35862----->it
24998----->s
61134----->fully
33914----->charged
37473----->but
35862----->it
24998----->s
33308----->good
2237----->for
28141----->about
34895----->5
10606----->hrs
200----->at
12868----->least
11367----->without
9415----->charging
12746--

In [22]:
def encode(text_tensor,label_tensor):
  encoded_text=encoder.encode(text_tensor.numpy())
  label=tf.where(label_tensor>3,1,0)
  return encoded_text,label

In [23]:
def encode_map_fn(tensor):
  text=tensor['data'].get('review_body')
  label=tensor['data'].get('star_rating')

  encoded_text,label=tf.py_function(encode,inp=[text,label],Tout=(tf.int64,tf.int32))

  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text,label

In [24]:
ar_encoded_data=train_dataset.map(encode_map_fn)

In [25]:
for f0,f1 in ar_encoded_data.take(2):
  print(f0)
  print(f1)

tf.Tensor(
[67446 13311   690 41220 15368 42316 30162 62045 47170 32918 34759 16010
 39590  5659 35862 37410 61161 16109 41239 20269 13248 28102 72849 62045
 23834 23522 20269 72013 34895 10553 13392 66158 57753 31458 19521 46646
 32918 37647 35364 34927 12814 23834 22787 19521  9031 19521 37647 52079
 60875 37647 17156 65236 66158 72647 19521 43497 50666 36611 70351 15009
 32425 25229 67055 61161 16109 52218 67055  6268 62053 25593  4827 37410
  6268 45745  6268 27966 35237 55287   690 33308 37908 25593 39590 67249
 52218 31256 71861 35862], shape=(88,), dtype=int64)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(
[23834 23522  9043 45127 24998  6268 57798  6268 27754 11550  1677  6862
  8189 36611 55551 10282 17028 23834 39060 35862 49800 20269  6704  6268
 34016 66048 35862 19521  6380], shape=(29,), dtype=int64)
tf.Tensor(1, shape=(), dtype=int32)


In [26]:
TAKE_SIZE=10000

train_data=ar_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data=train_data.padded_batch(BATCH_SIZE)


test_data=ar_encoded_data.take(TAKE_SIZE)
test_data=test_data.padded_batch(BATCH_SIZE)


In [27]:
vocab_size +=1

In [28]:
sample_text,sample_labels=next(iter(test_data))

sample_text[0],sample_labels[0]

(<tf.Tensor: shape=(300,), dtype=int64, numpy=
 array([67446, 13311,   690, 41220, 15368, 42316, 30162, 62045, 47170,
        32918, 34759, 16010, 39590,  5659, 35862, 37410, 61161, 16109,
        41239, 20269, 13248, 28102, 72849, 62045, 23834, 23522, 20269,
        72013, 34895, 10553, 13392, 66158, 57753, 31458, 19521, 46646,
        32918, 37647, 35364, 34927, 12814, 23834, 22787, 19521,  9031,
        19521, 37647, 52079, 60875, 37647, 17156, 65236, 66158, 72647,
        19521, 43497, 50666, 36611, 70351, 15009, 32425, 25229, 67055,
        61161, 16109, 52218, 67055,  6268, 62053, 25593,  4827, 37410,
         6268, 45745,  6268, 27966, 35237, 55287,   690, 33308, 37908,
        25593, 39590, 67249, 52218, 31256, 71861, 35862,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,  

In [29]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,128))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)))

for units in [64,64]:
  model.add(tf.keras.layers.Dense(units,activation='relu'))

model.add(tf.keras.layers.Dense(1))

In [30]:
!rm-r /tmp/logs/

/bin/bash: rm-r: command not found


In [31]:
logdir=os.path.join('/tmp/logs',datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
tensorboard_callbacks=tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
checkpointer=tf.keras.callbacks.ModelCheckpoint(filepath='tmp/sentiment_analysis.hdf5',verbose=1,save_best_only=True)


model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
history=model.fit(train_data,epochs=4,validation_data=test_data,callbacks=[tensorboard_callbacks,checkpointer])

Epoch 1/4
208/742 [=======>......................] - ETA: 1:51:52 - loss: 0.6433 - accuracy: 0.5933